# Encoding comparison

This notebook contains a side-by-side evaluation of different bit-to-tree encodings

In [1]:
import sys
sys.path.append('../')

import plotly.graph_objects as go
import numpy as np
import math
import random

from tree_lib.encodings import two_choices

#### 1 - Define encodings to compare

In [2]:
ENCODINGS = [["two_choices", two_choices.bits_to_tree, two_choices.tree_to_bits]]

#### 2. Check for correctness

In [7]:
# Auto-correlation with lag 1
def gen_signal(length, auto_correlation):
    assert 0 <= auto_correlation <= 1 and length > 0
    first_bit = f"{random.randint(0, 1)}"
    signal = ["0"]*length
    signal[0] = first_bit
    for i in range(1,length):
        signal[i] = signal[i-1] if random.random() < auto_correlation else f"{1-int(signal[i-1])}"
    return ''.join(signal)
    
STRINGS_TO_CHECK = [
    "0","1","01","10","11", "100", "101", "110", "111",
    "010101010101", "0000000000", "1111111111", 
    "010101010101"*100, "0000000000"*100, "1111111111"*100, 
]
STRINGS_TO_CHECK += [gen_signal(1000, random.random()) for _ in range(1000)]

for name, btt, ttb in ENCODINGS:
    for original_string in STRINGS_TO_CHECK:
        decoded = ttb(btt(original_string))
        assert decoded == original_string, f"'{name}' failed: '{original_string}' != '{decoded}'"
    print(f"{name} OK")


two_choices OK


#### Performance of the algorithm